## Detect

In [ ]:
# from ultralytics import YOLO
from PIL import Image
import cv2
from ultralytics.models.yolo.model import YOLO
import torch

model = YOLO("checkpoints/comic-speech-bubble-detector.pt", task='detect')
model.export(format='onnx', imgsz=640)  # creates 'yolov8n.onnx'

# results = model.predict(source='GarakutayaManta', save=True, device='cuda:0')  # save plotted images

In [ ]:
from ultralytics import YOLO
from PIL import Image
import cv2
import pickle

model = YOLO("checkpoints/comic-speech-bubble-detector-640.onnx", task='detect')

results = model.predict(source='dataset/test_folder', device='cpu') 

In [ ]:
result_info = dict()

for i, result in enumerate(results):
    coords = result.boxes.xyxy
    sub_dict = dict()
    for j, coord in enumerate(coords):
        info_dict = dict()
        xmin, ymin, xmax, ymax = coord
        info_dict['coord'] = (int(xmin), int(ymin), int(xmax), int(ymax))
        info_dict['img'] = result.orig_img[int(ymin):int(ymax), int(xmin):int(xmax)]
        sub_dict[j] = info_dict
    result_info['img'] = result.orig_img
    result_info[i] = sub_dict

file_path = 'nested_dictionary.pkl'

# Save the nested dictionary to a pickle file
with open(file_path, 'wb') as file:
    pickle.dump(result_info, file)

In [ ]:
file_path = 'output.pkl'
with open(file_path, 'rb') as file:
    loaded_dict = pickle.load(file)

print("Loaded dictionary:", loaded_dict)


In [ ]:
import numpy as np
from PIL import Image
from matplotlib import cm

image_path = 'dataset/GarakutayaManta/000.jpg'
# img = cv2.imread(image_path)
# img = cv2.resize(img, (1024, 1024))
# # convert to 3x1024x1024
# img = np.moveaxis(img, -1, 0)
model(image_path)

In [ ]:
from pathlib import Path

a[0].save_crop('', file_name=Path('im'))
a[0].save_txt('crop.txt')

## Visualize label

In [ ]:
from bs4 import BeautifulSoup
import os

comic = "AisazuNihaIrarenai"

label = f"dataset/Manga109_released_2023_12_07/annotations/{comic}.xml"
images = f"dataset/Manga109_released_2023_12_07/images/{comic}"
outputs = f"outputs/{images.split('/')[-1]}"

os.makedirs(outputs, exist_ok=True)
images_list = sorted(os.listdir(images))

with open(label, "r") as f:
    data = f.read()

bs_data = BeautifulSoup(data, 'xml')

for page in bs_data.find_all('page'):
    page_index = page['index']
    print(f"Page Index: {page_index}")

    image = cv2.imread(os.path.join(images, images_list[int(page_index)]))

    # Find all face, frame, and body elements within the page
    elements = page.find_all(['text'])
    for element in elements:
        xmin = element['xmin']
        ymin = element['ymin']
        xmax = element['xmax']
        ymax = element['ymax']
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
    cv2.imwrite(os.path.join(outputs, images_list[int(page_index)]), image)
